In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

#got here: https://stackoverflow.com/questions/34398054/ipython-notebook-cell-multiple-outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#ml stuff 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score

In [27]:
matches = pd.read_csv('matches.csv', low_memory=False)

matches.head()
rows = len(matches.axes[0])
cols = len(matches.axes[1])
print("Number of Rows: ", rows)
print("Number of Columns: ", cols)
matches.columns.values

,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,game_status,shootout,home_score,away_score,referee,home_goal_minutes,home_goal_scorers,away_goal_minutes,away_goal_scorers,home_starting_1_num,home_starting_1,home_starting_2_num,home_starting_2,home_starting_3_num,home_starting_3,home_starting_4_num,home_starting_4,home_starting_5_num,home_starting_5,home_starting_6_num,home_starting_6,home_starting_7_num,home_starting_7,home_starting_8_num,home_starting_8,home_starting_9_num,home_starting_9,home_starting_10_num,home_starting_10,home_starting_11_num,home_starting_11,home_bench_1_num,home_bench_1,home_bench_1_minute,home_bench_2_num,home_bench_2,home_bench_2_minute,home_bench_3_num,home_bench_3,home_bench_3_minute,home_bench_4_num,home_bench_4,home_bench_4_minute,home_bench_5_num,home_bench_5,home_bench_5_minute,home_bench_6_num,home_bench_6,home_bench_6_minute,home_bench_7_num,home_bench_7,home_bench_7_minute,home_bench_8_num,home_bench_8,home_bench_8_minute,home_bench_9_num,home_bench_9,home_bench_9_minute,away_starting_1_num,away_starting_1,away_starting_2_num,away_starting_2,away_starting_3_num,away_starting_3,away_starting_4_num,away_starting_4,away_starting_5_num,away_starting_5,away_starting_6_num,away_starting_6,away_starting_7_num,away_starting_7,away_starting_8_num,away_starting_8,away_starting_9_num,away_starting_9,away_starting_10_num,away_starting_10,away_starting_11_num,away_starting_11,away_bench_1_num,away_bench_1,away_bench_1_minute,away_bench_2_num,away_bench_2,away_bench_2_minute,away_bench_3_num,away_bench_3,away_bench_3_minute,away_bench_4_num,away_bench_4,away_bench_4_minute,away_bench_5_num,away_bench_5,away_bench_5_minute,away_bench_6_num,away_bench_6,away_bench_6_minute,away_bench_7_num,away_bench_7,away_bench_7_minute,away_bench_8_num,away_bench_8,away_bench_8_minute,away_bench_9_num,away_bench_9,away_bench_9_minute,home_bench_10_num,home_bench_10,home_bench_10_minute,home_starting_12_num,home_starting_12,home_starting_13_num,home_starting_13,home_starting_14_num,home_starting_14,away_starting_12_num,away_starting_12,away_starting_13_num,away_starting_13,away_starting_14_num,away_starting_14,away_starting_15_num,away_starting_15,home_starting_15_num,home_starting_15,home_starting_16_num,home_starting_16,home_starting_17_num,home_starting_17,home_starting_18_num,home_starting_18,home_starting_19_num,home_starting_19,home_starting_20_num,home_starting_20,home_starting_21_num,home_starting_21,home_starting_22_num,home_starting_22,home_bench_11_num,home_bench_11,home_bench_11_minute,home_bench_12_num,home_bench_12,home_bench_12_minute,home_bench_13_num,home_bench_13,home_bench_13_minute,home_bench_14_num,home_bench_14,home_bench_14_minute,away_starting_16_num,away_starting_16,away_starting_17_num,away_starting_17,away_starting_18_num,away_starting_18,away_starting_19_num,away_starting_19,away_starting_20_num,away_starting_20,away_starting_21_num,away_starting_21,away_bench_10_num,away_bench_10,away_bench_10_minute,away_bench_11_num,away_bench_11,away_bench_11_minute,away_bench_12_num,away_bench_12,away_bench_12_minute,away_starting_22_num,away_starting_22,away_bench_13_num,away_bench_13,away_bench_13_minute,home_possessionPct,away_possessionPct,home_shotsSummary,away_shotsSummary,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,away_bench_14_num,away_bench_14,away_bench_14_minute,home_formation,away_formation
0,NaN,New England,San Jose,7/31/1996,1996,NaN,"12,871",Foxboro Stadium,1996 MLS,Regular Season,FT,NaN,2,0,Ruben Rodhas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

Number of Rows:  7289
Number of Columns:  209


array(['id', 'home', 'away', 'date', 'year', 'time (utc)', 'attendance',
       'venue', 'league', 'part_of_competition', 'game_status',
       'shootout', 'home_score', 'away_score', 'referee',
       'home_goal_minutes', 'home_goal_scorers', 'away_goal_minutes',
       'away_goal_scorers', 'home_starting_1_num', 'home_starting_1',
       'home_starting_2_num', 'home_starting_2', 'home_starting_3_num',
       'home_starting_3', 'home_starting_4_num', 'home_starting_4',
       'home_starting_5_num', 'home_starting_5', 'home_starting_6_num',
       'home_starting_6', 'home_starting_7_num', 'home_starting_7',
       'home_starting_8_num', 'home_starting_8', 'home_starting_9_num',
       'home_starting_9', 'home_starting_10_num', 'home_starting_10',
       'home_starting_11_num', 'home_starting_11', 'home_bench_1_num',
       'home_bench_1', 'home_bench_1_minute', 'home_bench_2_num',
       'home_bench_2', 'home_bench_2_minute', 'home_bench_3_num',
       'home_bench_3', 'home_bench_3_min

In [91]:
#will create our predicator variable - result that has 3 options (0 for lost, 1 for win, 2 for draw)
matches['result'] = 0
matches.loc[(matches['home'] == 'San Jose Earthquakes') & (matches['home_score'] > matches['away_score']), 'result'] = 1
matches.loc[(matches['away'] == 'San Jose Earthquakes') & (matches['away_score'] > matches['home_score']), 'result'] = 1
matches.loc[(matches['home'] == 'San Jose Earthquakes') & (matches['home_score'] == matches['away_score']), 'result'] = 2
matches.loc[(matches['away'] == 'San Jose Earthquakes') & (matches['away_score'] == matches['home_score']), 'result'] = 2

# Filter the DataFrame to include matches involving the San Jose Earthquakes
san_jose_matches = matches[(matches['home'] == 'San Jose Earthquakes') | (matches['away'] == 'San Jose Earthquakes')]

# Select the desired columns
sj_matches_subset = san_jose_matches[['home', 'away', 'home_score', 'away_score', 'result']]

# Print the subset DataFrame
print(sj_matches_subset.head())

# convert these to numeric
san_jose_matches['home_possessionPct'] = san_jose_matches['home_possessionPct'].str.replace('%', '').astype(float)
san_jose_matches['away_possessionPct'] = san_jose_matches['away_possessionPct'].str.replace('%', '').astype(float)

# nan values, just give it 0
san_jose_matches = san_jose_matches.fillna(0)


                      home                    away  home_score  away_score  \
978   San Jose Earthquakes               LA Galaxy           3           2   
984              FC Dallas    San Jose Earthquakes           1           0   
989                  Miami    San Jose Earthquakes           2           1   
996   Sporting Kansas City    San Jose Earthquakes           0           0   
1002  San Jose Earthquakes  New England Revolution           3           2   

      result  
978        1  
984        0  
989        0  
996        0  
1002       1  


C:\Users\xfuen\AppData\Local\Temp\ipykernel_8564\1473037804.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  san_jose_matches['home_possessionPct'] = san_jose_matches['home_possessionPct'].str.replace('%', '').astype(float)
C:\Users\xfuen\AppData\Local\Temp\ipykernel_8564\1473037804.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  san_jose_matches['away_possessionPct'] = san_jose_matches['away_possessionPct'].str.replace('%', '').astype(float)


In [92]:
#this one does win, draw, and lost

# Split the data into features and target variables
X = san_jose_matches[['home_score', 'away_score', 'home_possessionPct', 'away_possessionPct', 'home_foulsCommitted', 'away_foulsCommitted', 'home_redCards', 'away_redCards', 'home_wonCorners', 'away_wonCorners', 'home_saves', 'away_saves']]
y_result = san_jose_matches['result']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_result, test_size=0.2, random_state=42)

model = LogisticRegression(multi_class='ovr')
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


C:\Users\xfuen\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='ovr')

Accuracy: 0.43410852713178294


In [96]:
# Print classification report
print("Classification Report:")
print(classification_report(y_test, predictions))  # Change average parameter to 'weighted'


Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.33      0.39        48
           1       0.36      0.56      0.44        41
           2       0.57      0.42      0.49        40

    accuracy                           0.43       129
   macro avg       0.46      0.44      0.44       129
weighted avg       0.46      0.43      0.43       129



In [39]:

matches = pd.read_csv('matches.csv')

matches['result'] = 0
matches.loc[(matches['home'] == 'San Jose Earthquakes') & (matches['home_score'] > matches['away_score']), 'result'] = 1
matches.loc[(matches['away'] == 'San Jose Earthquakes') & (matches['away_score'] > matches['home_score']), 'result'] = 1
matches.loc[(matches['home'] == 'San Jose Earthquakes') & (matches['home_score'] == matches['away_score']), 'result'] = 2
matches.loc[(matches['away'] == 'San Jose Earthquakes') & (matches['away_score'] == matches['home_score']), 'result'] = 2

sj_matches = matches[(matches['home'] == 'San Jose Earthquakes') | (matches['away'] == 'San Jose Earthquakes')]

sj_matches = sj_matches[sj_matches['result'] != 2]

sj_matches_subset = sj_matches[['home', 'away', 'home_score', 'away_score', 'result']]
print(sj_matches_subset.head())

sj_matches['home_possessionPct'] = sj_matches['home_possessionPct'].str.replace('%', '').astype(float)
sj_matches['away_possessionPct'] = sj_matches['away_possessionPct'].str.replace('%', '').astype(float)

sj_matches = sj_matches.fillna(0)

X = sj_matches[['home_score', 'away_score', 'home_possessionPct', 'away_possessionPct', 'home_foulsCommitted', 'away_foulsCommitted', 'home_redCards', 'away_redCards', 'home_saves', 'away_saves']]
y_result = sj_matches['result']


X_train, X_test, y_train, y_test = train_test_split(X, y_result, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

print(classification_report(y_test, predictions))

                      home                    away  home_score  away_score  \
978   San Jose Earthquakes               LA Galaxy           3           2   
984              FC Dallas    San Jose Earthquakes           1           0   
989                  Miami    San Jose Earthquakes           2           1   
1002  San Jose Earthquakes  New England Revolution           3           2   
1008  San Jose Earthquakes               DC United           3           2   

      result  
978        1  
984        0  
989        0  
1002       1  
1008       1  


C:\Users\xfuen\AppData\Local\Temp\ipykernel_10452\1037258721.py:1: DtypeWarning: Columns (11,14,50,102,123,125,129,131,133,135,137,139,141,143,145,147,149,157,158,160,161,163,165,167,169,171,173,184,186,187,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  matches = pd.read_csv('matches.csv')
C:\Users\xfuen\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Accuracy: 0.5851063829787234
              precision    recall  f1-score   support

           0       0.55      0.64      0.59        44
           1       0.63      0.54      0.58        50

    accuracy                           0.59        94
   macro avg       0.59      0.59      0.59        94
weighted avg       0.59      0.59      0.58        94

